## Evaluation
Submissions are evaluated on Root-Mean-Squared-Error (RMSE) between the logarithm of the predicted value and the logarithm of the observed sales price. (Taking logs means that errors in predicting expensive houses and cheap houses will affect the result equally.)

## Blueprint

## 1. Data Exploration

### - Import packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from scipy.special import expit
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
import shap
shap.initjs()

# Show all the columns and rows
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

### - Load dataset

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
id_train = train.Id
train = train.drop(columns=["Id"])

print(train.shape)
train.head()

(1460, 80)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
id_test = test.Id
test = test.drop(columns=["Id"])

print(test.shape)
test.head()

(1459, 79)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


### - Training data split

In [15]:
y = train['SalePrice']
y = y.astype('category').cat.codes
X_cat = train.drop('SalePrice', axis=1)

In [17]:
# Fit encoder
enc = OneHotEncoder()
enc.fit(X_cat)

# Transform categorical features
X_encoded = enc.transform(X_cat).toarray()

# Create feature matrix
feature_names = X_cat.columns
new_feature_names = enc.get_feature_names(feature_names)

X = pd.DataFrame(X_encoded, columns= new_feature_names)

In [19]:
print(X.shape)
X.head()

(1460, 7648)


,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotFrontage_21.0,LotFrontage_24.0,LotFrontage_30.0,LotFrontage_32.0,LotFrontage_33.0,LotFrontage_34.0,LotFrontage_35.0,LotFrontage_36.0,LotFrontage_37.0,LotFrontage_38.0,LotFrontage_39.0,LotFrontage_40.0,LotFrontage_41.0,LotFrontage_42.0,LotFrontage_43.0,LotFrontage_44.0,LotFrontage_45.0,LotFrontage_46.0,LotFrontage_47.0,LotFrontage_48.0,LotFrontage_49.0,LotFrontage_50.0,LotFrontage_51.0,LotFrontage_52.0,LotFrontage_53.0,LotFrontage_54.0,LotFrontage_55.0,LotFrontage_56.0,LotFrontage_57.0,LotFrontage_58.0,LotFrontage_59.0,LotFrontage_60.0,LotFrontage_61.0,LotFrontage_62.0,LotFrontage_63.0,LotFrontage_64.0,LotFrontage_65.0,LotFrontage_66.0,LotFrontage_67.0,LotFrontage_68.0,LotFrontage_69.0,LotFrontage_70.0,LotFrontage_71.0,LotFrontage_72.0,LotFrontage_73.0,LotFrontage_74.0,LotFrontage_75.0,LotFrontage_76.0,LotFrontage_77.0,LotFrontage_78.0,LotFrontage_79.0,LotFrontage_80.0,LotFrontage_81.0,LotFrontage_82.0,LotFrontage_83.0,LotFrontage_84.0,LotFrontage_85.0,LotFrontage_86.0,LotFrontage_87.0,LotFrontage_88.0,LotFrontage_89.0,LotFrontage_90.0,LotFrontage_91.0,LotFrontage_92.0,LotFrontage_93.0,LotFrontage_94.0,LotFrontage_95.0,LotFrontage_96.0,LotFrontage_97.0,LotFrontage_98.0,LotFrontage_99.0,LotFrontage_100.0,LotFrontage_101.0,LotFrontage_102.0,LotFrontage_103.0,LotFrontage_104.0,LotFrontage_105.0,LotFrontage_106.0,LotFrontage_107.0,LotFrontage_108.0,LotFrontage_109.0,LotFrontage_110.0,LotFrontage_111.0,LotFrontage_112.0,LotFrontage_114.0,LotFrontage_115.0,LotFrontage_116.0,LotFrontage_118.0,LotFrontage_120.0,LotFrontage_121.0,LotFrontage_122.0,LotFrontage_124.0,LotFrontage_128.0,LotFrontage_129.0,LotFrontage_130.0,LotFrontage_134.0,LotFrontage_137.0,LotFrontage_138.0,LotFrontage_140.0,LotFrontage_141.0,LotFrontage_144.0,LotFrontage_149.0,LotFrontage_150.0,LotFrontage_152.0,LotFrontage_153.0,LotFrontage_160.0,LotFrontage_168.0,LotFrontage_174.0,LotFrontage_182.0,LotFrontage_313.0,LotFrontage_nan,LotArea_1300,LotArea_1477,LotArea_1491,LotArea_1526,LotArea_1533,LotArea_1596,LotArea_1680,LotArea_1869,LotArea_1890,LotArea_1920,LotArea_1936,LotArea_1950,LotArea_1953,LotArea_1974,LotArea_2001,LotArea_2016,LotArea_2117,LotArea_2160,LotArea_2217,LotArea_2268,LotArea_2280,LotArea_2289,LotArea_2308,LotArea_2368,LotArea_2448,LotArea_2500,LotArea_2522,LotArea_2544,LotArea_2572,LotArea_2592,LotArea_2628,LotArea_2645,LotArea_2651,LotArea_2665,LotArea_2887,LotArea_3000,LotArea_3010,LotArea_3013,LotArea_3072,LotArea_3136,LotArea_3180,LotArea_3182,LotArea_3196,LotArea_3230,LotArea_3316,LotArea_3363,LotArea_3378,LotArea_3500,LotArea_3600,LotArea_3604,LotArea_3635,LotArea_3636,LotArea_3675,LotArea_3684,LotArea_3696,LotArea_3735,LotArea_3842,LotArea_3880,LotArea_3922,LotArea_3950,LotArea_3951,LotArea_3964,LotArea_3982,LotArea_4017,LotArea_4043,LotArea_4045,LotArea_4058,LotArea_4060,LotArea_4118,LotArea_4130,LotArea_4224,LotArea_4230,LotArea_4251,LotArea_4270,LotArea_4274,LotArea_4280,LotArea_4282,LotArea_4388,LotArea_4400,LotArea_4403,LotArea_4426,LotArea_4435,LotArea_4438,LotArea_4456,LotArea_4500,LotArea_4571,LotArea_4590,LotArea_4608,LotArea_4671,LotArea_4712,LotArea_4750,LotArea_4800,LotArea_4920,LotArea_4923,LotArea_4928,LotArea_5000,LotArea_5001,LotArea_5062,LotArea_5063,LotArea_5100,LotArea_5105,LotArea_5119,LotArea_5232,LotArea_5250,LotArea_5271,LotArea_5306,LotArea_5310,LotArea_5330,LotArea_5350,LotArea_5362,LotArea_5381,LotArea_5389,LotArea_5395,LotArea_5400,LotArea_5436,LotArea_5500,LotArea_5520,LotArea_5586,LotArea_5587,LotArea_5600,LotArea_5604,LotArea_5664,LotArea_5684,LotArea_5687,LotArea_5700,LotArea_5720,LotArea_5784,LotArea_5790,LotArea_5814,LotArea_5820,LotArea_5825,LotArea_5868,LotArea_5890,LotArea_5900,LotArea_5925,LotArea_6000,LotArea_6040,LotArea_6060,L

In [20]:
# Train model
model = xgb.XGBClassifier(objective="binary:logistic", 
                         max_depth=2, 
                         n_estimators=10)
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=2, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=10, n_jobs=0,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, ...)

### - Predict

In [ ]:
# Fit encoder
enc2 = OneHotEncoder()
enc2.fit(test)

# Transform categorical features
X_test = enc2.transform(X_test).toarray()

# Create feature matrix
feature_names2 = X_test.columns
new_feature_names2 = enc2.get_feature_names(feature_names2)

X_test = pd.DataFrame(X_test, columns= new_feature_names2)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(type(y_pred))
y_pred.head()

### - Using SHAP

In [21]:
# Get SHAP values
explainer = shap.Explainer(model)
shap_values = explainer(X)

# Waterfall plot for first observation
shap.plots.waterfall(shap_values[0])

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


In [ ]:
print(shap_values)

In [ ]:
# Get number of unique categories for each feature 
n_categories = []
for feat in feature_names[:-1]:
    n = X_cat[feat].nunique()
    n_categories.append(n)

In [ ]:
new_shap_values = []
for values in shap_values.values:
    
    #split shap values into a list for each feature
    values_split = np.split(values , np.cumsum(n_categories))
    
    #sum values within each list
    values_sum = [sum(l) for l in values_split]
    
    new_shap_values.append(values_sum)

### - Check distribution

In [ ]:
train.SalePrice.hist(grid=True, bins=20, rwidth=0.9, color='#607c8e')

### - Investigate object features

In [ ]:
# Number of features which are not numeric
print(sum(train.dtypes==object), "features are not numeric.")

In [ ]:
cri = train.dtypes==object
object_feat = train[cri.index[cri]]

In [ ]:
# Average house price for each object features
def mean_median(f):
    print("Mean of", train.groupby(f).mean().SalePrice, "\n")
    print("Median of", train.groupby(f).median().SalePrice, "\n")

for f in list(object_feat.columns):
    mean_median(f)

In [ ]:
# Delete features with moderate distribution
def filtered_feat(features):
    l = []
    for f in features:
        diff = max(train.groupby(f).mean().SalePrice)-min(train.groupby(f).mean().SalePrice)
        if diff >= 50000:
            l.append(f)
    return l

len(filtered_feat(list(object_feat.columns)))   # 38 features are still alive

### - Numerize

### - Correlation

In [ ]:
# LotFrontage & LotArea

### - Find null values

## 2. 

## 3. 